# CSCI 3155: Spring 2025
# Recitation Week 8

### Closures and Let Rec

## Static and Dynamic Scope 
Last week we looked at simple function definitions and dynamic scoping.

Consider the example:

~~~ocaml
let x = 5 in 
   let addx = function(y) x + y in 
      let x = 10 in /
          addx(2)
          
~~~

Or equivalently in scala:

~~~scala
val x = 5;
{
    val addx: Int => Int = (y: Int) => { x+y }
    {
        val x = 10;
        {
            addx(2)
        }
    }
}
~~~

* What should the value of this program be:
    * Under dynamic scoping conditions?
        * ???
    * Under static scoping conditions?
        * ???
* Which scoping condition have we used more in programming languages (static or dynamic)?
    * ???

## Closures

A *closure* is a way to represent functions in our environments, containing the variable name, expression to be evaluated, and environment to execute the function in.

Why should a closure include an environment?

Consider the example:

~~~ocaml
let x = 5 in 
   let addx = function(y) x + y in 
      let x = 10 in 
          addx(2)
          
~~~

Or equivalently in scala:

In [1]:
// try out scala code after discussion
val x = 5;
{
    val addx: Int => Int = (y: Int) => { x+y }
    {
        val x = 10;
        {
            addx(2)
        }
    }
}

x: Int = 5
res1_1: Int = 7

* We see the value 7 for the full program
* Scala, like many languages uses **static scoping**
* How could we build an interpreter to have the same effect?

## Exercise 1: Currying a function

Today we only allow functions to have one argument each. Could we update the language to have multiple argument functions?

~~~ocaml
let foo = function(x)
            function(y)
                x + y in
    let tmp = foo(5) in
        tmp(10)
        
~~~

* What is the value of the above program?
    * `15`
    ```scala
    foo(x)(y)
    tmp = foo(5)(y)
    ```
* How could we rewrite it with a function with multiple arguments?

~~~ocaml
_YOUR_ANSWER_HERE_
???
~~~

## Exercise 2: Functions with Multiple Parameters

Here, we will explore extending our closures to support multi parameter functions.

For this exercise, we will use a small part of the grammar we have been working with and change our function calls and definitions.

Reference: https://github.com/sriram0339/csci3155_notebooks/blob/master/7/RecursionInLettuce.ipynb

$$
\begin{array}{c|c}
    \text{original grammar} & \text{updated multiparameter function grammar} \\\hline
    \begin{array}{rcl}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
    \end{array}
    &
    \begin{array}{rcll}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef(\color{red} {\mathbf{Identifier}^*}, \mathbf{Expr}) & \text{Note multiple parameters now possible} \\ 
     & | & FunCall(\mathbf{Expr}, \color{red} {\mathbf{Expr}^*}) & \text{function call - expr(expr1, ... , exprn)} \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
    \end{array}
\end{array}
$$

Write the scala definition for `FunDef` and `FunCall`. Please use lists to implement the Kleene Star.

#### Note: 

Kleene star, represented by the '$^*$', is a way of representing multiple instances of a something.
It could be one, two, ten, or even none of the item in question.
For example, $\mathbf{Number}^*$ could be no numbers, a single number like $10$, or several numbers, like $10,\ 21,\ 32$. In Scala, we use `List` datatype to implement Kleene star mechanics.

In [2]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr

//BEGIN SOLUTION
case class FunDef(params: List[String], e: Expr) extends Expr
case class FunCall(e: Expr, params: List[String]) extends Expr
//END SOLUTION

case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Let
defined class IfThenElse
defined class Geq
defined class FunDef
defined class FunCall
defined class TopLevel

### Environment

We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered?

* The empty environment: implemented as an empty map. Let us call this environment: $EmptyEnv$
* The environment $\sigma[x \mapsto v]$ which denotes a previously existing environment $\sigma$ extended with the mapping $x \mapsto v$ that associates identifier $x$ with value $v$. Let us call this operation $\texttt{Extend}(\sigma, x, v)$.

### Closure
If you recall, last week we had something similar to Closure, which we called Function. Last week's implementation was based on dynamic scoping, now let's look at static scoping implementation.

We will now redefine closures for functions with zero or more args. When our function had one argument, our closures were defined as `Closure(id, expr, env)`. We would now like to define closures as
`Closure( [id1, ..., idn], expr, env)` where 
  - `id1..., idn` are the list of arguments for the function to be called. 
  - `expr` is the body of the function and 
  - `env` is the stored environment for static scoping.
 
 $$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & Num(\mathbf{Double}) \\
 & \Rightarrow & Closure(\color{red}{\mathbf{String}^*}, \mathbf{Expr}, \mathbf{Environment}) \\
 & \Rightarrow & Error \\
 \\
 \\
 \mathbf{Environment} & \Rightarrow & EmptyEnv \\
 & \Rightarrow & Extend(String, Value, Environment) \\
 \end{array}$$

In [3]:
sealed trait Environment 
sealed trait Value

case class Num(d: Double) extends Value
case class Bool(b: Boolean) extends Value
case object Error extends Value
//BEGIN SOLUTION
case class Closure(params: List[String], body: Value, env: Environment) extends Value
//END SOLUTION

case object EmptyEnv extends Environment // Represent an environment with nothing in it.
case class Extend(x: String, v: Value, sigma: Environment) extends Environment

defined trait Environment
defined trait Value
defined class Num
defined class Bool
defined object Error
defined class Closure
defined object EmptyEnv
defined class Extend

Build an interpreter using the following semantic rules. Ensure that your interpreter correctly deals with the cases that give rise to error by throwing an exception.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Closure}(\texttt{[id1,..., idk]}, \texttt{e}, \sigma)}{fundef}$$
<br /><br />

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\forall\ i \in \{ 1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = v_i,  v_i \not= \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \eval(\texttt{fBody}, \color{red}{\sigma_{cl}} \color{blue}{\circ [id1 \rightarrow v_1, \ldots, idk \rightarrow v_k]})}{funcall-ok}$$
 <br /><br />
          
Let us interpret this rule.
   - __Purpose__ : Evaluate an expression of the form `FunCall(e, [e1,...,ek])` where `e` is the expr for the called function, and `e1, ..., ek` are exprs for the arguments of this call. There are $k$ arguments.
     - The called function `e` must evaluate to a closure of the form $\text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \sigma_{cl})$.
     - The function call must have the same number of arguments as the closure ($n=k$).
     - Each of the $k$ arguments `ei` must evaluate to a value $v_i$ where $v_i$ is not error.
     - Then the result of evaluating the function call is the same as that of evaluating `fBody` under the environment $\sigma_{cl}$ extended by mapping the formal parameters of the closure, `id1,.., idk`, to $v_1, \ldots, v_k$, respectively.


We can also write some error rules:

$$\semRule {\eval(\texttt{e}, \sigma) \not\in \text{Closure}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-not-a-function}$$
<br /><br />


$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n \not= k}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-wrong-num-args}$$
<br /><br />
           
  
$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\exists\ i \in \{1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-arg-error}$$
<br /><br />
           
           
**Hint:** Can map, zip, and foldLeft be used for this?
 

In [4]:
class ErrorException(s:String) extends Exception(s){}

def lookupEnv(sigma: Environment, x: String): Value = sigma match {
    case EmptyEnv => throw new ErrorException(s"Error could not find string $x in environment")
    case Extend(y, v, rest) => {
        // BEGIN SOLUTION
        if (x == y) v
        else lookupEnv(rest, x)
        // END SOLUTION
    }
}

defined class ErrorException
defined function lookupEnv

In [5]:
val env0 = EmptyEnv
val env1 = Extend("x", Num(5), env0)
val env2 = Extend("foo", Closure(List("y", "z"),
                                 Plus(Plus(Ident("x"), Ident("y")), Ident("z")), 
                                 env1), 
                  env1)
val env3 = Extend("x", Num(10), env2)

// lookupEnv(env0, "x") // is an error
// lookupEnv(env0, "y") // is an error
// lookupEnv(env0, "foo") // is an error

assert(lookupEnv(env1, "x") == Num(5), "test x in env1")
// lookupEnv(env1, "y") // is an error
// lookupEnv(env1, "foo") // is an error

assert(lookupEnv(env2, "x") == Num(5), "test x in env2")
// lookupEnv(env2, "y") // is an error
assert(lookupEnv(env2, "foo") == Closure(List("y", "z"),
                                 Plus(Plus(Ident("x"), Ident("y")), Ident("z")), 
                                 Extend("x", Num(5), EmptyEnv)), 
       "test foo in env2")

assert(lookupEnv(env3, "x") == Num(10), "test x in env3")
// lookupEnv(env3, "y") // is an error
assert(lookupEnv(env3, "foo") == Closure(List("y", "z"),
                                 Plus(Plus(Ident("x"), Ident("y")), Ident("z")), 
                                 Extend("x", Num(5), EmptyEnv)), 
       "test foo in env3")

cmd5.sc:4: type mismatch;
 found   : cmd5.this.cmd2.Plus
 required: cmd5.this.cmd3.Value
                                 Plus(Plus(Ident("x"), Ident("y")), Ident("z")), 
                                     ^
cmd5.sc:20: type mismatch;
 found   : cmd5.this.cmd2.Plus
 required: cmd5.this.cmd3.Value
                                 Plus(Plus(Ident("x"), Ident("y")), Ident("z")), 
                                     ^
cmd5.sc:27: type mismatch;
 found   : cmd5.this.cmd2.Plus
 required: cmd5.this.cmd3.Value
                                 Plus(Plus(Ident("x"), Ident("y")), Ident("z")), 
                                     ^
Compilation Failed

In [5]:
def valueToNumber(v: Value): Double = v match {
    case Num(d) => d
    case _ => throw new ErrorException(s"Could not convert value $v to a number")
}

def eval(e: Expr, env: Environment): Value = {
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(eval(e1, env))
        val v2 = valueToNumber(eval(e2, env))
        val v3 = fun(v1, v2)
        Num(v3)
    }  
    
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(eval(e1, env))
        val v2 = valueToNumber(eval(e2, env))
        val v3 = fun(v1, v2)
        Bool(v3)
    }
    
    e match {
        case Const(d) => Num(d)
        case Ident(x) => lookupEnv(env, x)
        case Plus(e1,e2) => applyArith2(e1, e2)(_ + _)
        case Minus(e1, e2) =>  applyArith2(e1, e2)(_ - _)
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        
        case Let(id, e1, e2) => {
            val v1 = eval(e1, env)
            val env2 =  Extend(id, v1, env)
            eval(e2, env2)
        }
        
        case IfThenElse(e1, e2, e3) => {
            eval(e1, env) match {
                case Bool(true) => eval(e2, env)
                case Bool(false) => eval(e3, env)
                case _ => throw new ErrorException("Not a bool for Geq operation")
            }
        }
        
        case FunDef(idList, e) => {
            // BEGIN SOLUTION
            Closure(idList, e, env)
            // END SOLUTION
        }
        
        /* the single parameter version of FunCall
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            // Since evaluating e2 did not fail with an exception,
            // if I reach this point in my execution, I know that
            // neither v1 nor v2 are error.
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        // */
        
        case FunCall(e, eList) => {
            //BEGIN SOLUTION
            // Hint: Can map, zip, and foldLeft be used for this?
            eval(FunCall, env) match {
                case Closure(idList, fBody, closureEnv) => {
                    if (eList.length != idList.length) throw new IllegalArgumentException(s"Expected ${idList.length} parameters in function call got ${eList.length}")
                    idList.zip(eList.map(x => eval(x, env))).foldLeft(closureEnv)(x => Extend(x._1, x._2, closureEnv))
                    
                }
            }
            //END SOLUTION
        }
        
    }
}


def evalProgram(p: Program): Value = p match {
    case TopLevel(e) => try
            eval(e, EmptyEnv)
    catch {
        case e: ErrorException => {
            println(e)
            Error
        }
        case e: IllegalArgumentException => {
            println(e)
            Error
        }
        case e => {
            println("Unknown Exception " + e.toString)
            Error
        }
            
    }
}

cmd5.sc:45: type mismatch;
 found   : cmd5.this.cmd2.Expr
 required: cmd5.this.cmd3.Value
            Closure(idList, e, env)
                            ^
cmd5.sc:71: type mismatch;
 found   : cmd5.this.cmd2.FunCall.type
 required: cmd5.this.cmd2.Expr
            eval(FunCall, env) match {
                 ^
cmd5.sc:74: type mismatch;
 found   : String
 required: cmd5.this.cmd2.Expr
                    idList.zip(eList.map(x => eval(x, env))).foldLeft(closureEnv)(x => Extend(x._1, x._2, closureEnv))
                                                   ^
cmd5.sc:97: This catches all Throwables. If this is really intended, use `case e : Throwable` to clear this warning.
        case e => {
             ^
Compilation Failed

### Tests

In [ ]:
//BEGIN TEST
// TEST three arguments
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo2")
// function(x, y,z)  x + y+z+z
val fun2 = FunDef(List("x", "y", "z"), Plus(Plus(Plus(x, y), z), z))
//let foo = function(x,y,z) x+y+z+z in foo(10, 20, 30)
val l2 = Let("foo2", fun2, FunCall(foo, List(Const(10.0), Const(20.0), Const(30.0))))
//Program
val p2 = TopLevel(l2)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(90.0), "Test 2 Failed")
//END TEST

In [ ]:
//BEGIN TEST
//Test zero arguments
/*
let x = 10 in
    let foo = function() x in
        foo()
*/
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List())
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p2 = TopLevel(l3)
val v3 = evalProgram(p2)
println(s"Your program evaluated to $v3")
assert(v3 == Num(10.0), "Test 3 Failed")
//END TEST

In [ ]:
//BEGIN TEST
/*
let x = 10 in
    let foo = function() x in
        foo(1)
*/
//Evaluate with wrong number of args
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List(Const(1.0)))
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p4 = TopLevel(l3)
val v4 = evalProgram(p4)
assert(v4 == Error, "Test 4 failed -- your program should have detected that arguments were mismatched")
//END TEST

In [ ]:
//BEGIN TEST
/*
let x = 1 in
    let z = 50 in
      let foo = fun(x,y) x + y + x + z in
        let = 2 in
          f(10,30)
*/
// TEST scoping
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo")
// function(x, y)  x + y + x + z 
val fun = FunDef(List("x", "y"), Plus(Plus(Plus(x, y), x),z))
val funcall = FunCall(foo, List(Const(10.0), Const(30.0)))
val l4 = Let("x", Const(1.0), Let("z", Const(50.0), Let("foo", fun, Let("y", Const(2.0), funcall))))
//Program
val p2 = TopLevel(l4)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(100.0), "Test 5 Failed == you did not evaluate static scoping correctly")
//END TEST

## Exercise 3: Recursive multi-param functions

Consider the below recursive function which takes multiple parameters: 

~~~
let rec crazy_rec = function (x, y)
                    if (x >= 0) then 0.5 + x + y
                    else 20 + crazy_rec (0 - x, y)
in 
crazy_rec( -5.0, 2.0)
~~~

We note here that, our grammar so far does not support $let rec$ as of now. Therefore, let us expand the above grammar with let recursion for functions with multiple parameters. 

$$
\begin{array}{c|c}
    \text{original grammar} & \text{updated multiparameter function grammar} \\\hline
    \begin{array}{rcl}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
     & | & \color{red}{LetRec(\mathbf{Identifier},\mathbf{Identifier} ,\mathbf{Expr}, \mathbf{Expr})}
    \end{array}
    &
    \begin{array}{rcll}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}^*, \mathbf{Expr}) & \text{Note multiple parameters now possible} \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}^*) & \text{function call - expr(expr1, ... , exprn)} \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
     & | & \color{red}{LetRec(\mathbf{Identifier},\mathbf{Identifier^*} ,\mathbf{Expr}, \mathbf{Expr})}
    \end{array}
\end{array}
$$

Now, we have to go back and modify some portions in $eval$ method and define the necessary case classes for making $LetRec$ with multiple parameters happen. 

In [ ]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class FunDef(idList: List[String], e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: List[Expr]) extends Expr

// BEGIN SOLUTION
???
// END SOLUTION 

case class TopLevel(e: Expr) extends Program

### Environment

We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered?

* The empty environment: implemented as an empty map. Let us call this environment: $EmptyEnv$
* The environment $\sigma[x \mapsto v]$ which denotes a previously existing environment $\sigma$ extended with the mapping $x \mapsto v$ that associates identifier $x$ with value $v$. Let us call this operation $\texttt{Extend}(\sigma, x, v)$.

#### ADDING
* Let us add a third kind of extension to support recursion:
    $\texttt{ExtendRec}(f, x, \texttt{e}, \sigma)$ which creates a new environment $\color{red}{\hat{\sigma}}$ such that $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$ and $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$.
    * NOTE: In looking up $\color{red}{\hat{\sigma}}(x)$ we can get a closure over $\color{red}{\hat{\sigma}}$ itself
    
 $$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & Num(\mathbf{Double}) \\
 & \Rightarrow & Closure(\color{red}{\mathbf{String}^*}, \mathbf{Expr}, \mathbf{Environment}) \\
 & \Rightarrow & Error \\
 \\
 \\
 \mathbf{Environment} & \Rightarrow & EmptyEnv \\
 & \Rightarrow & Extend(String, Value, Environment) \\
 & \Rightarrow & ExtendRec(String, \color{red}{String*}, Expr, Environment) \\
 \end{array}$$

In [ ]:
sealed trait Environment 
sealed trait Value

case class Num(d: Double) extends Value
case class Bool(b: Boolean) extends Value
case object Error extends Value
case class Closure(s: List[String], e: Expr, env: Environment ) extends Value

case object EmptyEnv extends Environment // Represent an environment with nothing in it.
case class Extend(x: String, v: Value, sigma: Environment) extends Environment

// Begin Solution
???
// End Solution

In [ ]:
class ErrorException(s:String) extends Exception(s){}  

def lookupEnv(sigma: Environment, x: String): Value = sigma match {
    case EmptyEnv => throw new ErrorException(s"Error could not find string $x in environment")
    case Extend(y, v, rest) => {
        if (y == x ) v
        else lookupEnv (rest, x)
    } 
    case ExtendRec(funName, params, funBody, rest) => {
        // BEGIN SOLUTION
        // The illusion of circularity is here.
        ???
        // END SOLUTION
    }
}

In [ ]:
// let rec crazy_rec = function (x, y)
//                     if (x >= 0) then 0.5 + x + y
//                     else 20 + crazy_rec (0 - x, y)
// in 
// crazy_rec( -5.0, 2.0)
val eBody = IfThenElse(Geq(Ident("x"), 
                           Const(0)),
                       Plus(Plus(Const(0.5), 
                                 Ident("x")), 
                            Ident("y")),
                       Plus(Const(20), 
                            FunCall(Ident("crazy_rec"), 
                                    List(Minus(Const(0),
                                               Ident("x")),
                                         Ident("y")))))

val env0 = EmptyEnv
val env1 = ExtendRec("crazy_rec", 
                     List("x", "y"), 
                     eBody,
                     env0)
val env2 = Extend("x", Bool(true), env1)

// lookupEnv(env0, "x")  // is an error
// lookupEnv(env0, "y")  // is an error
// lookupEnv(env0, "crazy_rec")  // is an error

// lookupEnv(env1, "x")  // is an error
// lookupEnv(env1, "y")  // is an error
// NOTE: tempting to say env0 in the result...
assert(lookupEnv(env1, "crazy_rec") == Closure(List("x", "y"), eBody, env1), "test crazy_rec in env1")

assert(lookupEnv(env2, "x") == Bool(true), "test x in env 2")
// lookupEnv(env2, "y")  // is an error
assert(lookupEnv(env2, "crazy_rec") == Closure(List("x", "y"), eBody, env1), "test crazy_rec in env2")

In [ ]:
def valueToNumber(v: Value): Double = v match {
    case Num(d) => d
    case _ => throw new ErrorException(s"Could not convert value $v to a number")
}

def eval(e: Expr, env: Environment): Value = {
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(eval(e1, env))
        val v2 = valueToNumber(eval(e2, env))
        val v3 = fun(v1, v2)
        Num(v3)
    }  
    
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(eval(e1, env))
        val v2 = valueToNumber(eval(e2, env))
        val v3 = fun(v1, v2)
        Bool(v3)
    }
    
    e match {
        case Const(d) => Num(d)
        case Ident(x) => lookupEnv(env, x)
        case Plus(e1,e2) => applyArith2(e1, e2)(_ + _)
        case Minus(e1, e2) =>  applyArith2(e1, e2)(_ - _)
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        
        case Let(id, e1, e2) => {
            val v1 = eval(e1, env)
            val env2 =  Extend(id, v1, env)
            eval(e2, env2)
        }
        
        case IfThenElse(e1, e2, e3) => {
            eval(e1, env) match {
                case Bool(true) => eval(e2, env)
                case Bool(false) => eval(e3, env)
                case _ => throw new ErrorException("Not a bool for Geq operation")
            }
        }
        
        case FunDef(idList, e) => {
            Closure(idList, e, env)
        }
        case FunCall(e, eList) => {
            val v = eval(e, env)
            v match {
                case Closure(argList, fBody, closureEnv) => {
                    if (argList.length == eList.length){
                        val valList = eList.map {eval(_, env)}
                        val zipList = argList.zip(valList)
                        val newMap = zipList.foldLeft[Environment] (env) { (newEnv, newBind) => Extend(newBind._1, newBind._2, newEnv) }
                        eval(fBody, newMap)
                    } else {
                        throw new ErrorException(s"Argument length mismatch")
                    }
                }
                case _ => throw new ErrorException(s"Tried to use a numerical value as a function")
            }
        }
        
        // Space for LetRec
        // BEGIN SOLUTION
        ???
        // END SOLUTION
    }
}


def evalProgram(p: Program): Value = p match {
    case TopLevel(e) => try
            eval(e, EmptyEnv)
    catch {
        case e: ErrorException => {
            println(e)
            Error
        }
        case e: IllegalArgumentException => {
            println(e)
            Error
        }
        case e => {
            println("Unknown Exception " + e.toString)
            Error
        }
            
    }
}

### Workspace
Space to add your logic here so that grammar reference becomes easy. Once done copy paste your implementation to above.

// Define LetRec
~~~
???
~~~

// Define Extend Rec
~~~
???
~~~

// Space for Letrec eval logic
~~~
???
~~~

#### Test

Run this test after you are done with making changes as per the new grammar above. 

In [ ]:
/**
let rec crazy_rec = function(x, y)
        if ( x >= 0 )
        then 0.5 + x + y
        else 22 + crazy_rec( 0-x, y)
    in
    crazy_rec(-5, 2)
 */
val crec = Ident("crazy_rec")
val x = Ident("x")
val y = Ident("y")

val e1 = FunCall(crec, List(Const(-5.0), Const(2.0)))
val e21 = Plus(Const(0.5), Plus(x, y))
val e22 = Plus(Const(20.0), FunCall(crec, List(Minus(Const(0), x), y)))
val e2 = LetRec("crazy_rec", List("x", "y"), IfThenElse(Geq(x, Const(0)), e21, e22), e1)

val crazyProg = TopLevel(e2)
assert (evalProgram(crazyProg) == Num(27.5))

## That's all folks !